In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb

from alpha_vantage.timeseries import TimeSeries
# from sqlalchemy import create_engine
ts = TimeSeries(key='35RWNNYNVVW23JS1', output_format='pandas')

In [ ]:
ET, meta_data = ts.get_daily(symbol='ET', outputsize='full')
# 🔹 Rename columns for clarity
ET.columns = ["Open", "High", "Low", "Close", "Volume"]

# 🔹 Convert index to datetime format
ET.index = pd.to_datetime(ET.index)

# 🔹 Convert index (date) into a regular column
ET.reset_index(inplace=True)

# Calculate 100-day moving average for Close price
ET["100d_MA"] = ET["Close"].rolling(window=100).mean()

# 🔹 Rename the date column
ET.rename(columns={"date": "Date"}, inplace=True)
ET['Date'].max()

In [ ]:

# Calculate the moving average and standard deviation
ET['bb_mean'] = ET['Close'].rolling(window=20).mean()  # 20-day SMA
ET['bb_std'] = ET['Close'].rolling(window=20).std()   # 20-day standard deviation

# Calculate the Bollinger Bands
ET['bb_upper'] = ET['bb_mean'] + (ET['bb_std'] * 2)
ET['bb_lower'] = ET['bb_mean'] - (ET['bb_std'] * 2)
ET['Mean'] = ET['Close'].mean()
ET['Median'] = ET['Close'].median()
ET.head()

In [ ]:
con = duckdb.connect(database='./vangaurd.duckdb', read_only=False)

In [ ]:
con.register('temp_df', ET)  # Register the DataFrame as a virtual table
con.execute("DROP TABLE IF EXISTS ET")
con.execute("CREATE TABLE ET AS SELECT * FROM temp_df")  # Create the table
con.unregister('temp_df') #unregister the temp table.

# con.close()

# BND


In [ ]:
BND, meta_data = ts.get_daily(symbol='BND', outputsize='full')
# 🔹 Rename columns for clarity
BND.columns = ["Open", "High", "Low", "Close", "Volume"]

# 🔹 Convert index to datetime format
BND.index = pd.to_datetime(BND.index)

# 🔹 Convert index (date) into a regular column
BND.reset_index(inplace=True)

# Calculate 100-day moving average for Close price
BND["100d_MA"] = BND["Close"].rolling(window=100).mean()

# 🔹 Rename the date column
BND.rename(columns={"date": "Date"}, inplace=True)
BND['Date'].max()

In [15]:

# Calculate the moving average and standard deviation
BND['bb_mean'] =BND['Close'].rolling(window=20).mean()  # 20-day SMA
BND['bb_std'] =BND['Close'].rolling(window=20).std()   # 20-day standard deviation

# Calculate the Bollinger Bands
BND['bb_upper'] = BND['bb_mean'] + (BND['bb_std'] * 2)
BND['bb_lower'] = BND['bb_mean'] - (BND['bb_std'] * 2)
BND['Mean'] = BND['Close'].mean()
BND['Median'] = BND['Close'].median()
BND.head()

,Date,Open,High,Low,Close,Volume,100d_MA,bb_mean,bb_std,bb_upper,bb_lower,Mean,Median
0,2025-03-28,73.16,73.3500,73.07,73.32,4850495.0,NaN,NaN,NaN,NaN,NaN,80.095279,81.05
1,2025-03-27,72.88,72.9200,72.82,72.90,5568322.0,NaN,NaN,NaN,NaN,NaN,80.095279,81.05
2,2025-03-26,73.02,73.0591,72.89,72.93,4881976.0,NaN,NaN,NaN,NaN,NaN,80.095279,81.05
3,2025-03-25,73.00,73.1699,72.99,73.09,6133739.0,NaN,NaN,NaN,NaN,NaN,80.095279,81.05
4,2025-03-24,73.19,73.2200,72.98,73.00,5637341.0,NaN,NaN,NaN,NaN,NaN,80.095279,81.05


In [16]:
con.register('temp_df', BND)  # Register the DataFrame as a virtual table
con.execute("DROP TABLE IF EXISTS BND")
con.execute("CREATE TABLE BND AS SELECT * FROM temp_df")  # Create the table
con.unregister('temp_df') #unregister the temp table.

# con.close()

# VTI

In [17]:
VTI, meta_data = ts.get_daily(symbol='VTI', outputsize='full')
# 🔹 Rename columns for clarity
VTI.columns = ["Open", "High", "Low", "Close", "Volume"]

# 🔹 Convert index to datetime format
VTI.index = pd.to_datetime(VTI.index)

# 🔹 Convert index (date) into a regular column
VTI.reset_index(inplace=True)

# Calculate 100-day moving average for Close price
VTI["100d_MA"] = VTI["Close"].rolling(window=100).mean()

# 🔹 Rename the date column
VTI.rename(columns={"date": "Date"}, inplace=True)
VTI['Date'].max()

Timestamp('2025-03-28 00:00:00')

In [21]:

# Calculate the moving average and standard deviation
VTI['bb_mean'] = VTI['Close'].rolling(window=20).mean()  # 20-day SMA
VTI['bb_std'] = VTI['Close'].rolling(window=20).std()   # 20-day standard deviation

# Calculate the Bollinger Bands
VTI['bb_upper'] = VTI['bb_mean'] + (VTI['bb_std'] * 2)
VTI['bb_lower'] = VTI['bb_mean'] - (VTI['bb_std'] * 2)
VTI['Mean'] = VTI['Close'].mean()
VTI['Median'] = VTI['Close'].median()
VTI.head()

,Date,Open,High,Low,Close,Volume,100d_MA,bb_mean,bb_std,bb_upper,bb_lower,Mean,Median
0,2025-03-28,278.31,278.75,273.05,273.43,3912899.0,NaN,NaN,NaN,NaN,NaN,127.55734,113.3
1,2025-03-27,279.46,281.08,278.05,278.99,2345342.0,NaN,NaN,NaN,NaN,NaN,127.55734,113.3
2,2025-03-26,284.15,284.92,280.31,281.16,2473462.0,NaN,NaN,NaN,NaN,NaN,127.55734,113.3
3,2025-03-25,284.52,285.09,283.66,284.49,2517248.0,NaN,NaN,NaN,NaN,NaN,127.55734,113.3
4,2025-03-24,282.25,284.53,281.90,284.01,3251647.0,NaN,NaN,NaN,NaN,NaN,127.55734,113.3


In [22]:
con.register('temp_df', VTI)  # Register the DataFrame as a virtual table
con.execute("DROP TABLE IF EXISTS VTI")
con.execute("CREATE TABLE VTI AS SELECT * FROM temp_df")  # Create the table
con.unregister('temp_df') #unregister the temp table.

# con.close()

# BNDX

In [ ]:
BNDX, meta_data = ts.get_daily(symbol='BNDX', outputsize='full')
# 🔹 Rename columns for clarity
BNDX.columns = ["Open", "High", "Low", "Close", "Volume"]

# 🔹 Convert index to datetime format
BNDX.index = pd.to_datetime(BNDX.index)

# 🔹 Convert index (date) into a regular column
BNDX.reset_index(inplace=True)

# Calculate 100-day moving average for Close price
BNDX["100d_MA"] = BNDX["Close"].rolling(window=100).mean()

# 🔹 Rename the date column
BNDX.rename(columns={"date": "Date"}, inplace=True)
BNDX['Date'].max()

In [ ]:
con.register('temp_df', BNDX)  # Register the DataFrame as a virtual table
con.execute("DROP TABLE IF EXISTS BNDX")
con.execute("CREATE TABLE BNDX AS SELECT * FROM temp_df")  # Create the table
con.unregister('temp_df') #unregister the temp table.

# con.close()

# VXUS

In [ ]:
VXUS, meta_data = ts.get_daily(symbol='VXUS', outputsize='full')
# 🔹 Rename columns for clarity
VXUS.columns = ["Open", "High", "Low", "Close", "Volume"]

# 🔹 Convert index to datetime format
VXUS.index = pd.to_datetime(VXUS.index)

# 🔹 Convert index (date) into a regular column
VXUS.reset_index(inplace=True)

# Calculate 100-day moving average for Close price
VXUS["100d_MA"] = VXUS["Close"].rolling(window=100).mean()

# 🔹 Rename the date column
VXUS.rename(columns={"date": "Date"}, inplace=True)
VXUS['Date'].max()

In [ ]:
con.register('temp_df', VXUS)  # Register the DataFrame as a virtual table
con.execute("DROP TABLE IF EXISTS VXUS")
con.execute("CREATE TABLE VXUS AS SELECT * FROM temp_df")  # Create the table
con.unregister('temp_df') #unregister the temp table.

con.close()